In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import torch
from random import choice
import math
import gym
import copy
import time
import shutil
import tensorflow as tf
import tensorflow.compat.v1 as tf
import time
import gym
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [ ]:
path2 = 'C:\\Users\\raven\\MAP-Elites-NEAT'

In [ ]:
os.chdir(path2)

## Create Graph files

In [ ]:
def create_graphs(num_models,num_inp,num_out):
    #remove all files in Graph_folder
    if(os.getcwd()+'\\Graph_folder' in [x[0] for x in os.walk(os.getcwd())]):
        shutil.rmtree(os.getcwd()+'\\Graph_folder\\')
        
    #create new file
    os.mkdir(os.getcwd()+'\\Graph_folder\\')
    
    for i in range(num_models):
        #create number of files needed
        fh=open(os.getcwd()+"\\Graph_folder\\model_"+str(i)+".edgelist", "x")
        fh.close()

## Read model from file

In [ ]:
def read_graph(model_num,num_inp,num_out):
    fh=open(os.getcwd()+"\\Graph_folder\\model_"+str(model_num)+".edgelist", "rb")
    G = nx.read_edgelist(fh,nodetype=int,create_using=nx.DiGraph(),data=(('weight',float),('activation',int),('inn_num',int)))
    fh.close() 
    
    #add input and output nodes
    for i in range(num_inp+num_out):
        G.add_node(i)
    return G

## Save Model

In [ ]:
def write_graph(model,model_num):
    nx.write_edgelist(model, os.getcwd()+"\\Graph_folder\\model_"+str(model_num)+".edgelist",data=['weight','activation','inn_num'])

## Mutate

In [ ]:
def mutate(gene_model,num_models,max_inn_num,model_num):
    new_con_prob = .5
    con_to_node_prob = .2
    change_weights = .8
    
    if(random.random() < new_con_prob):
        add_con(gene_model,num_models,max_inn_num,model_num)
        
    if(random.random() < con_to_node_prob):
        con_to_node(gene_model,num_models,max_inn_num,model_num)
    
    if(random.random() < change_weights):
        new_weight(gene_model,model_num)    

## Add Connection

In [ ]:
def add_con(model,num_models,max_inn_num,model_num):
    node_list = list(model.nodes)
    
    #get two random nodes
    length = len(node_list)
    node1 = node_list[random.randint(0,length-1)]
    node2 = node_list[random.randint(num_out,length-1)]
    
    #check if node2 is input node, node1 is an output node, or nodes are the same
    if( (node2<num_inp) or (node1>=num_inp and node1<num_inp+num_out) or (node1 == node2)):
        return 0

    #check if adding edge will cause loop
    if(check_edges(model,node1,node2)):
        model.add_edge(node1,node2,weight=random.random(),activation=1,inn_num=inn_num((node1,node2),num_models,max_inn_num))
        max_inn_num[0]+=1

## Connection to Node

In [ ]:
def con_to_node(model,num_models,max_inn_num,model_num):
    #if no connections, dont run rest of function
    temp = list(model.edges())
    if(len(temp)==0):
        return 0
    
    node1,node2 = temp[random.randint(0,len(temp)-1)]

    new_node = max(list(model.nodes))+1

    w1 = 1
    w2 = nx.get_edge_attributes(model,'weight').get((node1,node2))

    model.add_edge(node1,new_node,weight=w1,activation=1)
    model.add_edge(new_node,node2,weight=w2,activation=1)
    model.edges[node1,node2]['activation'] = 0
    
    model.edges[node1,new_node]['inn_num'] = inn_num((node1,new_node),num_models,max_inn_num)
    model.edges[new_node,node2]['inn_num'] = inn_num((new_node,node2),num_models,max_inn_num)

## Change weight

In [ ]:
def new_weight(model,model_num):
    for edge in model.edges():
        if(random.random()<.1):
            #change weight
            weight = nx.get_edge_attributes(model,'weight').get(edge)
            model.edges[edge[0], edge[1]]['weight'] = random.random()
        else:
            #perturb weight
            weight = nx.get_edge_attributes(model,'weight').get(edge)
            model.edges[edge[0], edge[1]]['weight'] = model.edges[edge[0], edge[1]]['weight'] + random.random()*.2            

## Innovation numbers

In [ ]:
def inn_num(edge,num_models,max_inn_num):
    #check if edge exists, if it does, set edge innovation number to match
    for model_num in range(num_models):
        model = read_graph(model_num,num_inp,num_out) 
        temp_dict = nx.get_edge_attributes(model,'inn_num')
        if edge in temp_dict.keys():
            return temp_dict[edge]
    
    #if new edge, then return current unused max innovation number
    max_inn_num[0]+=1
    return max_inn_num[0]-1

## Create TensorFlow graph

In [ ]:
def get_tf_model(model,num_inp,num_out):
    #Create calc_dict of input nodes
    calc_dict = {}
    for i in range(num_inp): calc_dict[i]=tf.placeholder(tf.float32,[1])
        
    #get list of uncalculated nodes            
    temp_uncalc_node_list = []
    for i in model.nodes():
        if(i not in calc_dict):
            temp_uncalc_node_list.append(i)

    #calculate every node in uncalclated node list until list is empty
    index = -1
    while(len(temp_uncalc_node_list)>0):
        index+=1
        node = temp_uncalc_node_list[index]
        
        #if any input node is uncalculated, break, else, calculate current node
        temp_list = list(model.in_edges(node))
        temp_bool = False
        for i in temp_list:
            if i[0] not in calc_dict:
                temp_bool = True
                break
        if(temp_bool):
            continue

        #sum over all edge weight multiplied by the child nodes output
        tensor_sum = tf.zeros([1],dtype=tf.float32)
        for cur_edge in temp_list:
            weight = nx.get_edge_attributes(model,'weight').get(cur_edge)
            weight = tf.constant(weight,dtype=tf.float32)

            activation = nx.get_edge_attributes(model,'activation').get(cur_edge)
            activation = tf.constant(activation,dtype=tf.float32)

            node_output = calc_dict[cur_edge[0]]
            tensor_sum = node_output*weight*activation + tensor_sum

        #add calculated node to calc_dict and remove node from uncalc_node list
        calc_dict[node] = tf.math.tanh(tensor_sum)
        temp_uncalc_node_list.pop(index)
        index = -1
                
    #return list of output nodes
    out_list = []
    for i in range(num_inp,num_inp+num_out):
        if(i not in calc_dict):
            #not all output nodes are accounted for and are missing in calc_dict
            print("!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!")
        else:
            out_list.append(calc_dict[i])

    return out_list,calc_dict

## Speciate

In [ ]:
#problem with c2 here
def speciate(dict_rew_list,num_models,c1,c2,c3):
    threshold = 4.0
    max_species = 0
    
    #double for loop, 
    #create list of model_numbers with the same species
    #[ [0,1,2,3], [4], [5,6,7,8], [9]]   Here, there are 4 species with 10 networks
    used_model_num_list = [i for i in range(num_models)]
    spec_list = [[(dict_rew_list[0],0)]] #initialized with the first model in its own species
    max_species = 0
    for model_num2 in used_model_num_list:
        found_species = False
        for species in spec_list:
            model_num1 = species[0][1]
            #if model number are the same, skip
            if(model_num1 == model_num2):
                found_species = True
                break
            
            #read the two models and calculate the difference
            gene_model1 = read_graph(model_num1,num_inp,num_out)
            gene_model2 = read_graph(model_num2,num_inp,num_out)
            diff = model_dif(gene_model1,gene_model2,c1,c2,c3)
            #print("diff: {}".format(diff))
            #if difference is below threshold, set model1 species to model2 species
            if(diff < threshold):
                #find graph in species lists
                species.append((dict_rew_list[model_num2],model_num2))
                found_species = True
                break
        if(found_species==False):
            spec_list.append([(dict_rew_list[model_num2],model_num2)])
            
    return spec_list

## Speciate helper function (Adjusted fitness)

In [ ]:
def model_dif(gene_model1,gene_model2,c1,c2,c3):
    delta_sum = 0
    num_disjoint = 1
    num_excess = 1
    num_weights = 1
    weight_sum = 0
    
    temp_dict1 = nx.get_edge_attributes(gene_model1,'weight')
    temp_dict2 = nx.get_edge_attributes(gene_model2,'weight')
    for edge in temp_dict1:
        if(edge in temp_dict2):
            #weight difference
            num_weights+=1
            weight1 = nx.get_edge_attributes(gene_model1,'weight').get(edge)
            weight2 = nx.get_edge_attributes(gene_model2,'weight').get(edge)
            weight_sum += abs(weight2-weight1)
        else:
            # disjoint
            num_disjoint+=1
    
    for edge in temp_dict2:
        #excess
        if(edge not in temp_dict1):
            num_disjoint+=1
        
    delta_sum = (c1*num_excess)/(num_excess + num_disjoint) + (c2*num_disjoint)/(num_excess + num_disjoint) + (c3*weight_sum)/num_weights
    return delta_sum

## Crossover

In [ ]:
def crossover(reward_list,num_models,max_inn_num,keep_amt,c1,c2,c3):
    #need to do speciation before deletion
    temp_reward_list = [(sub[1], sub[0]) for sub in reward_list] 
    dict_rew_list = dict(temp_reward_list)
    
    #speciate
    spec_list = speciate(dict_rew_list,num_models,c1,c2,c3)
    print(len(spec_list))
    
    #adjust fitness
    #get average reward of population
    avg_rew = 0
    for i in reward_list:
        avg_rew += i[0]
    avg_rew = avg_rew
    
    #get number of graphs per species
    num_children_list = []
    for spec in spec_list:
        per_children = 0
        for model_rew,model_num in spec:
            per_children += model_rew
            
        per_children = per_children/avg_rew
        num_children_list.append(per_children)
  
    #call delete graphs
    del_amt = int((1-keep_amt)*num_models)
    unused_model_num_list,used_model_num_list,del_rew_list = del_graphs(temp_reward_list,num_models,del_amt,spec_list,num_children_list)   
    
    unused_model_num_list.sort()
    used_model_num_list.sort()
    
    #crossover
    new_num_models = len(used_model_num_list)
    while(len(unused_model_num_list)>0):
        #choose two random and different indeces
        rand_ind1 = random.randint(0,len(used_model_num_list)-1)
        rand_ind2 = choice([i for i in range(0,len(used_model_num_list)-1) if i not in [rand_ind1]])
        
        #get model numbers
        model_num1 = used_model_num_list[rand_ind1]
        model_num2 = used_model_num_list[rand_ind2]
        
        #get models from model numbers        
        gene_model1 = read_graph(model_num1,num_inp,num_out)        
        gene_model2 = read_graph(model_num2,num_inp,num_out)
        
        #get new child model, write model, and pop used model number from list
        child_gene_model = get_child_gene_model(gene_model1,gene_model2,num_models,max_inn_num)
        #need to get innovation number of child gene
        write_graph(child_gene_model,unused_model_num_list[0])
        added_num = unused_model_num_list.pop(0)        

## Crossover helper function (Delete models)

In [ ]:
def del_graphs(reward_list,num_models,del_amt,spec_list,num_children_list):
    unused_model_num_list = []
    used_model_num_list = []
    del_rew_list = []
    path = os.getcwd()+"\\Graph_folder\\"
    
    spec_count = -1
    for perc in num_children_list:
        spec_count+=1
        spec_del_amt = int(perc*del_amt)
        spec_rew_list = spec_list[spec_count]
        spec_rew_list.sort()
        model_count = -1
        for i in spec_list[spec_count]:
            model_count+=1
            if(model_count < spec_del_amt):
                os.remove(path+"model_"+str(spec_rew_list[model_count][1])+".edgelist")     
                unused_model_num_list.append(spec_rew_list[model_count][1])
            else:
                used_model_num_list.append(spec_rew_list[model_count][1])
                del_rew_list.append(spec_rew_list[model_count])

    return unused_model_num_list,used_model_num_list,del_rew_list

## Get child model given parents

In [ ]:
def get_child_gene_model(gene_model1,gene_model2,num_models,max_inn_num):
    act_prob = .7
    child_model = nx.DiGraph()
    
    #get attributes of each parent graph
    weight_dict1 = nx.get_edge_attributes(gene_model1,'weight')
    weight_dict2 = nx.get_edge_attributes(gene_model2,'weight')
    act_dict1 = nx.get_edge_attributes(gene_model1,'activation')
    act_dict2 = nx.get_edge_attributes(gene_model2,'activation')
    inn_dict1 = nx.get_edge_attributes(gene_model1,'inn_num')
    inn_dict2 = nx.get_edge_attributes(gene_model2,'inn_num')
    
    #loop over first parent to see if nodes 
    for edge in weight_dict1:
        #if edge is in both models, 50% chance to choose either
        weight = 0
        activation = 1
        innovation = 0
        if(edge in weight_dict2):
            if(random.random() < .5):
                weight = weight_dict1.get(edge)
                activation = act_dict1.get(edge)
                innovation = inn_dict1.get(edge)
            else:
                weight = weight_dict2.get(edge)
                activation = act_dict2.get(edge)
                innovation = inn_dict2.get(edge)
        else:
            weight = weight_dict1.get(edge)
            activation = act_dict1.get(edge)
            innovation = inn_dict1.get(edge)
            
        #set activation to 1 with probability
        if(activation == 0):
            if(random.random()<act_prob):
                activation = 1
                
        #check if edge can be added without creating a circuit
        if(check_edges(child_model,edge[0],edge[1])):
            child_model.add_edge(edge[0],edge[1],weight=weight,activation=activation,inn_num=innovation)
        
    for edge in weight_dict2:
        #if edge is in both models, 50% chance to choose either
        if(edge not in weight_dict1):
            weight = 0
            activation = 1
            weight = weight_dict2.get(edge)
            activation = act_dict2.get(edge)
            innovation = inn_dict2.get(edge)
            
        #set activation to 1 with probability
        if(activation == 0):
            if(random.random()<act_prob):
                activation = 1
            
        #check if edge can be added without creating a circuit
        if(check_edges(child_model,edge[0],edge[1])):
            child_model.add_edge(edge[0],edge[1],weight=weight,activation=activation,inn_num=innovation)  
    return child_model

## Check for loops in graph

In [ ]:
def check_edges(model,node1,node2):
    if((node2,node1) in model.edges() or (node1,node2) in model.edges()):
        return False
            
    temp_model = model.copy()
    temp_model.add_edge(node1,node2,weight=random.randrange(-2,2))
    
    try:
        #if model has cycles, switch node1 and node2
        nx.find_cycle(temp_model, orientation='original')
    except:
        #model.add_edge(node1,node2,weight=random.random(),activation=1,inn_num=inn_num((node1,node2),num_models,max_inn_num))
        return True
    return False

## Hyper-parameters

In [ ]:
num_inp = 24
num_out = 4

#num_inp = 4
num_bias = 4
num_inp = num_inp + num_bias
#num_out = 1

#changing num_models limits amount of saved models: fix this
num_models = 100
trials = 1
num_game_steps = 500
#max_inn_num = [num_inp+num_out]
max_inn_num = [0]
gens = 1

c1 = 3.0
c2 = 3.0
c3 = 3.0

## Run networks

In [ ]:
#horizontal speed has index [2]
#legs touching groun has index [-11]

tf.reset_default_graph()
env = gym.make('BipedalWalker-v3')
create_graphs(num_models,num_inp,num_out)
generations = 20
keep_amt = .3

with tf.Session() as sess:
    for gen in range(generations):
        total_a = time.time()

        #empty reward
        reward_list = []

        #run over all models
        for model_num in range(num_models):
            print("model num: {}".format(model_num))
            #get model from file
            gene_model = read_graph(model_num,num_inp,num_out)
            
            #Mutation
            mutate(gene_model,num_models,max_inn_num,model_num)

            ##get model from file after mutations
            #gene_model = read_graph(model_num,num_inp,num_out)

            #turn gene model into tensorflow model
            tf_outputs,calc_dict = get_tf_model(gene_model,num_inp,num_out)
            
            #heiristic initializations
            avg_horizontal_speed = 0
            perc_legs_on_ground = 0
            
            #a = time.time()
            full_sum = 0
            for trial in range(trials):
                #get inputs
                observation = env.reset()
                observation = np.append(observation,[1])

                #run model
                model_r_sum = 0
                for game_steps in range(num_game_steps):
                    #set up feed dictionary for model input
                    feed_dict = {}
                    for i in range(len(observation)): feed_dict[calc_dict[i]] = [observation[i]]
                    for i in range(len(observation),num_inp): feed_dict[calc_dict[i]] = [1.0]
                    
                    actions = sess.run(tf_outputs,feed_dict=feed_dict)  #this is what costs the most time
                    
                    action = np.array([actions[i][0] for i in range(num_out)])
                    
                    #take action
                    observation, reward, done, info = env.step(action)

                    #add observation to heiristics
                    avg_horizontal_speed += observation[2]
                    perc_legs_on_ground += observation[-11]
                    
                    #append reward
                    model_r_sum+=reward
                    if(done):
                        env.close()
                        break

                full_sum+=model_r_sum
            reward_list.append((full_sum/trials,model_num))
            
        #Crossover
        crossover(reward_list,num_models,max_inn_num,keep_amt,c1,c2,c3)

        #calculate average reward
        sum_rew = 0
        for i in reward_list:
            sum_rew+= i[0]
        avg_rew = sum_rew/num_models
        #reward_list.append(avg_rew)

        total_b = time.time()
        print("Generation: {} Average reward: {} Time: {} Highest Reward: {}".format(gen,avg_rew,total_b-total_a,max(reward_list)))
sess.close()
total_b = time.time()

In [ ]:
#Fixes some weird tensorflow error
tf.compat.v1.disable_eager_execution()

#get model number
#reward_list.sort(reverse=True)
#model_num = reward_list[0][1]
#print(model_num)
model_num = 18

#get model from file
gene_model = read_graph(model_num,num_inp,num_out)


#get model from file after mutations
gene_model = read_graph(model_num,num_inp,num_out)

#turn gene model into tensorflow model
tf_outputs,calc_dict = get_tf_model(gene_model,num_inp,num_out)


#get inputs
observation = env.reset()
observation = np.append(observation,[1])

#run model
rewards = 0
with tf.Session() as sess:
    for game_steps in range(500):#num_game_steps):
        #set up feed dictionary for model input
        feed_dict = {}
        for i in range(len(observation)): feed_dict[calc_dict[i]] = [observation[i]]
        for i in range(len(observation),num_inp): feed_dict[calc_dict[i]] = [1.0]

        fin_out = []
        actions = sess.run(tf_outputs,feed_dict=feed_dict)  #this is what costs the most time

        for i in range(num_out):
            fin_out.append(actions[i][0])

        #take action
        env.render()
        action = np.array(fin_out)
        observation, reward, done, info = env.step(action)
        rewards+=reward
        if(done):
            break
print(rewards)